# IT Security - Sheet 4 "Authentication and Key Agreement"

**Total achievable points: 20**

**Released: 05.12.2025**

**Submission Deadline: 12.12.2025 13:00**

---
Groupnumber: 17

Names and matriculation numbers of **ALL** team members: 

Neo Ahrens (456647)

Christian Bick (456513)

Yorck Heilmann (456599)

---

**Important Information**

The assignments have to be submitted by groups of 4 students. Even if you are registered in RWTHmoodle to a submission group, **please include the group number as well as the name and matriculation number of every group member in this notebook**. In case you are not part of a submission group and want to hand in assignments, please contact `ba-itsec@itsec.rwth-aachen.de`.

Enter your solutions for the tasks in the respective cells of this notebook. These cells are either marked by "YOUR ANSWER HERE" or `#YOUR CODE HERE`. Do not add any new cells or remove existing ones, especially do not copy cells. Cells marked with `###PLAYGROUND` can be used to test your implementation and generate output (see example for the first tasks). Do not add any other output or tests in the cell of the task, just implement the function with the header provided. If you want to test your implementation, use the `###PLAYGROUND` cells. They will be ignored during grading. **Do not change any other cells or add new ones.**

All assertions provided by us should be considered part of the exercise description. Passing the assertions does not automatically mean that you'll get full points for the corresponding task, but failing the assertions pretty much guarantees that you'll get no points. 

**Do not import any further Python packages** except the default Python ones and the ones that are explicitly given by us.

## 0. Setup (0 points)
*This assignment has one big storyline that spans Tasks 1 and 2, but all tasks can still be solved independently. Furthermore, Task 1 is a large task, but Tasks 1.4 -- 1.5 can be solved independently of Tasks 1.1 -- 1.3, if you ignore the fact that the story makes more sense if you read the Task descriptions of Tasks 1.1 -- 1.3*.

Sometime, the story and the task are separated in case you want to skip the story. This is done as follows:

*Story*

---

*Task*

In this exercise, you will have to analyze certificates. We recommend using OpenSSL to do so, as it is a generally useful tool to troubleshoot certificate issues, which you will likely have to do at some point in your professional career (even if that career does not IT-security at all). Alternatively, you can also use a browser to inspect certificates.

We also provide a function to inspect certificates, which uses the *cryptography* package. If you are using the same environment as for the previous exercise, you should have installed it already. Otherwise, you can likely install it by uncommenting and executing the cell below. Note that we will not provide tech support if the cell fails to install the package for whatever reason.

In [111]:
### PLAYGROUND
#import sys
#!{sys.executable} -m pip install cryptography

## 1. Breaking into "Definetly High-seKurity Enterprises" (12 points)

Very recently, you were approached by a charismatic but morally ambiguous entrepreneur named Freddy Fraudster. He proposed a daring Ocean's 11 style heist, with the target being "Definetly High-seKurity Enterprises" (short DHKE). Said company has gathered and hoarded masses of data on individuals worldwide, and uses that data to operate bot-farms and manipulate elections. They are a fundamental threat to democracies all around the world, and the only way to stop them is exposing their malicious schemes. Also, breaking in could net you a hefty payday.

![Definetly High-seKurity Enterprises logo; generated with AI](dhke.png)
*Generated using AI*

A very important of your plan to expose DHKE, and to perform an "involuntary redistribution" of their assets, involves passing a security checkpoint. You will need a checkpoint code that changes every hour to get through this checkpoint. Prior reconnaissance has already revealed that every hour, the guard on duty receives a call from the headquarters through a landline, and that the checkpoint code is changed afterwards.

After staging a fake protest to cause a distraction, you and your team were able to install an active wiretap, that is able to listen and record but also is able to intercept and modify messages. You intently listen for the checkpoint code to pass through, but all you hear is... random noise? You dig deeper, breach the Git-server of the company and discover the following program used to generate the checkpoint codes.

---

*TLDR: Two parties, a headquarters and a guard, use the following program to generate "checkpoint codes":*

In [112]:
color_code_A = '\x1b[1;31m'
color_code_B = '\x1b[1;32m'
color_end = '\x1b[0m'

from random import randint

class Headquarters:
    g: int
    p: int
    __private_key: int
    code: int

    def __init__(self, g: int, p: int) -> None:
        self.g = g
        self.p = p

    def generate_message_for_guard(self) -> int:
        self.__private_key = randint(1, (self.p-1)//2)
        return pow(self.g, self.__private_key, self.p)
    
    def receive_message_from_guard(self, msg: int) -> None:
        self.code = pow(msg, self.__private_key, self.p) % (10**6)
        print(f"{color_code_A}Headquarters:{color_end} The code is {self.code}.")
    
class Guard:
    g: int
    p: int

    code: int

    def __init__(self, g: int, p: int) -> None:
        self.g = g
        self.p = p

    def generate_response_for_headquarters(self, msg: int) -> int:
        secret_key = randint(1, (self.p-1)//2)
        self.code = pow(msg, secret_key, self.p) % (10**6)
        print(f"{color_code_B}Guard:{color_end} The code is {self.code}.")
        return pow(self.g, secret_key, self.p)
    

def normal_code_exchange():
    g = 2
    p = 100000127
    hq = Headquarters(g, p)
    guard = Guard(g, p)

    msg1 = hq.generate_message_for_guard()
    print(f"Headquarters to guard: {msg1}")
    msg2 = guard.generate_response_for_headquarters(msg1)
    print(f"Guard to Headquarters: {msg2}")
    hq.receive_message_from_guard(msg2)
    assert hq.code == guard.code

normal_code_exchange()
print("\n---- One hour later ----")
normal_code_exchange()

Headquarters to guard: 80246916
Guard: The code is 361763.
Guard to Headquarters: 99225305
Headquarters: The code is 361763.

---- One hour later ----
Headquarters to guard: 84544326
Guard: The code is 296559.
Guard to Headquarters: 3196123
Headquarters: The code is 296559.


### Task 1.1 Understanding the checkpoint code generation (1 point)

**Name** the underlying protocol presented in the lecture that is used to generate the checkpoint code. **Explain briefly** why the checkpoint code changes every time the protocol is executed.

Diffie-Hellman Key Exchange is the underlying protocol.

The checkpoint code changes every time to ensure the key freshness property.

### Task 1.2 Thief-in-the-Middle (2 points)

As you are the techie of your heist-team, you quickly notice that the public keys exchanged in the checkpoint code generation protocol are not authenticated at all. Hence, you can put your expensive active wiretap to good use, and replace the headquarter's public key with your own public key. This easily allows you to calculate the guard's checkpoint code.

**Implement** the function `intercept_message_from_hq_to_guard`, that receives the headquarter's message to the guard as input, and that produces a modified message that the guard will receive. Also **implement** the function `get_code_from_message_from_guard_to_hq` that receives the guard's response and that returns the guard's checkpoint code.

In [113]:
class Task1_2_Attacker:
    g: int
    p: int

    secret_key: int

    def __init__(self, g: int, p: int) -> None:
        self.g = g
        self.p = p

    def intercept_message_from_hq_to_guard(self, msg: int) -> int:
        # YOUR CODE HERE
        self.secret_key = randint(1, (self.p-1)//2)
        return pow(self.g, self.secret_key, self.p)

    def get_code_from_message_from_guard_to_hq(self, msg: int) -> int:
        # YOUR CODE HERE
        return pow(msg, self.secret_key, self.p) % (10**6)

In [114]:
### PLAYGROUND
# You can add your own testing logic here. Everything written here will be ignored during grading.

In [115]:
# Your code is not necessarily correct just because it passes this test.

def code_generation_with_active_attacker(g: int, p: int):
    hq = Headquarters(g, p)
    guard = Guard(g, p)
    attacker = Task1_2_Attacker(g, p)

    msg1 = hq.generate_message_for_guard()
    print(f"Headquarters to attacker: {msg1}")
    msg1_prime = attacker.intercept_message_from_hq_to_guard(msg1)
    print(f"Attacker to guard: {msg1_prime}")
    msg2 = guard.generate_response_for_headquarters(msg1_prime)
    print(f"Guard to attacker: {msg2}")

    recovered_code = attacker.get_code_from_message_from_guard_to_hq(msg2)
    print(f"{color_code_A}Attacker:{color_end} Recovered the code {recovered_code}")
    assert recovered_code == guard.code

    hq.receive_message_from_guard(msg2)

code_generation_with_active_attacker(2, 100000127)

Headquarters to attacker: 50755908
Attacker to guard: 3664641
Guard: The code is 241752.
Guard to attacker: 10244394
Attacker: Recovered the code 241752
Headquarters: The code is 451797.


In [116]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [117]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 1.3 The attack's properties (3 points)

#### a) (0.5 points)
**Briefly describe** why the guard's checkpoint code and the headquarter's checkpoint code now differ.

The attacker introduces two seperate communication channels, Attacker-Guard and Attacker-HQ

#### b) (1.5 points)

Is it possible to modify the guard's response such that the headquarter produces the same checkpoint code? **Justify** your answer **in detail**. 

As we have knowledge of the modulus of the HQ's message, we can keep generating new secret keys until the modulo of the messages (our message and HQ) are the "same".
--> essentially brute force (practically inefficient but works in theory)

#### c) (1 point)

How could the DHKE-company fix the vulnerability / prevent your attack? **Explain** how your fix addresses the vulnerability.

The DHKE-company must fix the vulnerability by implementing public key authentication to prevent the Man-in-the-Middle attack.

The Headquarters and the Guard would each possess a long-term private signing key and a publicly verifiable signing certificate.

The signature verification will fail if something in the corresponding message is messed with.

### Task 1.4 Hardened defenses (3 points)

Well, you did not really care about being super stealthy as long as your heist can progress quick enough. Hence, the DHKE-company already grew suspicious of some interference with its operations. This got even worse when you loudly proclaimed "IM IN!", while the DHKE's chief information security officer walked by your lair purely by coincidence. He then launched a large scale investigation, uncovering your wiretap and how you attacked their checkpoint code generation protocol.

The good news are that you managed to move your computers and Mate bottles to a different hideout, leaving just minutes before the police arrived. But there are bad news too: You moving your equipment gave the DHKE company enough time to upgrade their protocol. 

---

The headquarters and the guard now share a secret key used to prevent active modifications. The new program is as follows:

In [118]:
def create_mac(msg: int, key: bytes) -> bytes:
    import hmac
    import math
    msg_as_bytes = msg.to_bytes(length=math.ceil(msg.bit_length() / 8))
    return hmac.digest(key, msg_as_bytes, digest="SHA256")

class Headquarters:
    g: int
    p: int
    __pre_shared_key: bytes
    __short_term_private_key: int
    code: int

    def __init__(self, g: int, p: int, pre_shared_key: bytes) -> None:
        self.g = g
        self.p = p
        self.__pre_shared_key = pre_shared_key

    def generate_message_for_guard(self) -> tuple[int, bytes]:
        self.__short_term_private_key = randint(1, (self.p-1)//2)
        public_key = pow(self.g, self.__short_term_private_key, self.p)
        mac = create_mac(public_key, self.__pre_shared_key)
        return (public_key, mac)
    
    def receive_message_from_guard(self, msg: tuple[int, bytes]) -> None:
        public_key, mac = msg
        if mac != create_mac(public_key, self.__pre_shared_key):
            raise ValueError("Invalid MAC.")
        self.code = pow(public_key, self.__short_term_private_key, self.p) % (10**6)
        print(f"{color_code_A}Headquarters:{color_end} The code is {self.code}.")
    
class Guard:
    g: int
    p: int
    __pre_shared_key: bytes

    code: int

    def __init__(self, g: int, p: int, pre_shared_key: bytes) -> None:
        self.g = g
        self.p = p
        self.__pre_shared_key = pre_shared_key

    def generate_response_for_headquarters(self, msg: tuple[int, bytes]) -> tuple[int, bytes]:
        hq_public_key, hq_mac = msg
        if hq_mac != create_mac(hq_public_key, self.__pre_shared_key):
            raise ValueError("Invalid MAC.")

        guard_private_key = randint(1, (self.p-1)//2)
        self.code = pow(hq_public_key, guard_private_key, self.p) % (10**6)
        print(f"{color_code_B}Guard:{color_end} The code is {self.code}.")

        guard_public_key = pow(self.g, guard_private_key, self.p)
        return (guard_public_key, create_mac(guard_public_key, self.__pre_shared_key))
    

def hardened_code_exchange():
    import os

    g = 2
    p = 100000127
    pre_shared_key = os.urandom(16)
    hq = Headquarters(g, p, pre_shared_key)
    guard = Guard(g, p, pre_shared_key)

    msg1 = hq.generate_message_for_guard()
    print(f"Headquarters to guard: {msg1}")
    msg2 = guard.generate_response_for_headquarters(msg1)
    print(f"Guard to Headquarters: {msg2}")
    hq.receive_message_from_guard(msg2)
    assert hq.code == guard.code

hardened_code_exchange()

Headquarters to guard: (12930699, b'\x12f\xcb\x15\xcd\x07\xbf ^\xc0YA\xc0\xc1]U0\xf8\x9c7\x07\xea\xf6\xe0\xe2\xab\x02\xec/8\x9d1')
Guard: The code is 457881.
Guard to Headquarters: (78125309, b'W0\xd7\x10+@\x9e{\xa3"\xd7\xb4M\x1fS\x88\xe9\x0frX\xe3\xc5\x16\r2\xb3o\xe0!r*\xd4')
Headquarters: The code is 457881.


This protocol definitely is a tougher nut to crack, so a through analysis of its properties is appropriate. Hence, you decide go over the key exchange protocols properties presented in the lecture:

##### a) Key freshness (0.5 points)

Does the protocol guarantee key / checkpoint code freshness for the Guard, the Headquarters, both, or no one? **Briefly justify** your answer.

The protocol guarantees key/checkpoint code freshness for both the Guard and the Headquarters, because they each generate a new secret key for every execution.

##### b) Perfect forward secrecy (0.5 points)

Does the protocol provide perfect forward secrecy? **Briefly justify** your answer.

Yes, the protocol provides perfect forward secrecy, as the session key is derived independently of the long-term key.

Long-term key is only used for authentication.

##### c) Protection against known-key attacks (0.5 points)

Does the protocol provide protection against known-key / known-checkpoint-code attacks? **Briefly justify** your answer.

Yes, past session keys have no influence on future ones.

##### d) Implicit key authentication (0.5 points)

Does the protocol provide implicit key / checkpoint code authentication? **Briefly justify** your answer.

*Hint: You can assume that only the headquarter and the guard know the pre-shared key.*

Yes, the protocol provides implicit key authentication because the successful verification of the MAC confirms that only a party knowing the pre-shared key could have contributed to the shared key.

##### e) Explicit key authentication (1 point)

Does the protocol provide explicit key authentication? Does this change once a trusted DHKE-employee is able to pass the security checkpoint by stating the correct checkpoint code? **Briefly justify** your answer.

The protocol does not provide explicit key authentication, but this is functionally achieved once the guard physically checks that a trusted employee successfully knows and states the correct derived checkpoint code.

### Task 1.5 Group pressure (3 points)

Well, it seems like this is a fairly solid protocol. Without knowing the pre-shared key, there is not much you can do. But your team, and especially the leader Freddy Fraudster, is having none of it. Very loudly he yells: Oh goddamn, how hard can it be hire a competent tech guy? They always come up with some excuses like "It does not work like in the movies" and "No, a ladder does not help you to climb over a firewall". It really cannot be that hard!

The other team members join in:
* "Are you sure the pre-shared key is not 123456? Let's try the guard's mother's maiden name!"
* "Come on, you have cracked everything so far! They only added a burger or whatever that mac is supposed to be, that cannot be that difficult."
* "I think our techie just does not type fast enough. In the movies, they always type for five seconds before they decrypt the mainframe. I think we need to buy a mechanical keyboard for it to work."

You cave into the group pressure and look at everything one more time to make sure that you have not missed anything. And that is when you notice that you still have access to the DHKE Git server. Maybe, you can implant some kind of backdoor into the checkpoint code generation protocol. A closer inspection reveals that you can change the Diffie-Hellman group, i.e., you overwrite the `g` and `p` used to perform the Diffie-Hellman key exchange. This surely can be abused to calculate the checkpoint code, even if the protocol itself is sound.

---

**Implement** the function `generate_vulnerable_group` that returns a tuple `(g, p)`, where `p` is a prime number that is at least 26 bits long, and `g` is the generator used to generate the public keys. Make sure that the guard and the headquarter generate the same checkpoint code when they use the group in their checkpoint code generation protocol.

**Also implement** the function `recover_code` that returns the guard's checkpoint code only using the (unmodified) messages exchanged during the checkpoint code generation protocol, and your vulnerable group.

*Hints:*

* *There are multiple ways to solve this task, each solution involving varying degrees of brute forcing. It also is possible to solve this task without any brute force at all. Try to do so first. If you must apply brute force, make sure that your implementation runs in under 3 seconds and requires less than 100MB RAM.*
* *You can use the prime 100000127 in the brute-force-free solution.*
* *If you do not find a way to solve this task without brute force, look into attacks on the discrete logarithm first and use the fact that you dictate the group. The right combination of generator and prime can drastically reduce the (effective) key space to only a handful of keys.*

In [119]:
def generate_vulnerable_group() -> tuple[int, int]:
    # YOUR CODE HERE
    return (1,100000127)

def recover_code(msg_from_hq_to_guard: tuple[int, bytes], msg_from_guard_to_hq: tuple[int, bytes], g: int, p: int) -> int:
    # YOUR CODE HERE
    return 1

In [120]:
### PLAYGROUND
# You can add your own testing logic here. Everything written here will be ignored during grading.

In [121]:
# Your code is not necessarily correct just because it passes this one test.
def attack_on_hardened_protocol():
    import os

    g, p = generate_vulnerable_group()
    if p.bit_length() < 26:
        raise ValueError("The prime must be at least 26 bits long.")
    if not isPrime(p):
        raise ValueError("p must be a prime.")
    pre_shared_key = os.urandom(16)
    hq = Headquarters(g, p, pre_shared_key)
    guard = Guard(g, p, pre_shared_key)

    msg1 = hq.generate_message_for_guard()
    print(f"Headquarters to guard: {msg1}")
    msg2 = guard.generate_response_for_headquarters(msg1)
    print(f"Guard to Headquarters: {msg2}")
    hq.receive_message_from_guard(msg2)
    
    if guard.code != hq.code:
        raise ValueError("Guard and headquarters no longer have the same code. Most likely, you have produced an incorrect group.")

    recovered_code = recover_code(msg1, msg2, g, p)
    print(f"Recovered code: {recovered_code}")
    assert recovered_code == guard.code, "🚨 The recovered code did not match the guard's code and you were caught. 🚨"

    print("🥷🏻 The team successfully passed the checkpoint and finished the heist. 💰💰💰")

def isPrime(n, k=5): 
    # Miller-Rabin primality test
    # This code is stolen from https://stackoverflow.com/a/36525088 because I do not want you to install yet another library.
    # While it is possible to craft specific "non-primes" that bypass this test (such that this check wrongfully thinks that the non-prime is a prime),
    # and such non-primes indeed lead to insecure or even incorrect Diffie-Hellman key exchanges, you can perform a much much simpler attack in this task.
    from random import randint
    if n < 2: return False
    for p in [2,3,5,7,11,13,17,19,23,29]:
        if n % p == 0: 
            return n == p
    s, d = 0, n-1
    while d % 2 == 0:
        s, d = s+1, d >> 1
    for i in range(k):
        x = pow(randint(2, n-1), d, n)
        if x == 1 or x == n-1: continue
        for r in range(1, s):
            x = (x * x) % n
            if x == 1: 
                return False
            if x == n-1: break
        else: 
            return False
    return True

attack_on_hardened_protocol()

Headquarters to guard: (1, b"VX?\xef\xd0\x19'\x8a\xb6\x88<\x14\xe9\x9b\xab\xadR\xee\xc1]g\xa2\x8d\xd8v\xe4r\xc2\xa2\xf3)1")
Guard: The code is 1.
Guard to Headquarters: (1, b"VX?\xef\xd0\x19'\x8a\xb6\x88<\x14\xe9\x9b\xab\xadR\xee\xc1]g\xa2\x8d\xd8v\xe4r\xc2\xa2\xf3)1")
Headquarters: The code is 1.
Recovered code: 1
🥷🏻 The team successfully passed the checkpoint and finished the heist. 💰💰💰


In [122]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

With this new and successful attack, you watch the hacked security feed as Freddy Fraudster and the others go through the security checkpoint. As you gave them the correct checkpoint code, the guard does not suspect anything. Next, they distract a secretary and slip into an elevator down to floor -10, where the vault is located. Now is the time to shine for two team members you have not yet paid attention to: Lerry Longfinger starts picking the lock. "Nothing on one, two is binding, one click, now we dropped into a deep false set, five is binding, nice click on three ...". You swear you have heard that voice somewhere already.
But you lack the time to think about it further as Lerry already has opened the vault. Athlete Alice then flies through the air as she dodges all the lazers and disables the alarm system before the temperature sensor notices her presence.

While the rest of the teams rushes to put all the money into the bags while Freddy Fraudster uploads the data from the one single server theatrically positioned in the center of the vault. You stare at the stream of incoming data, with document viewers opening rapidly. But something does not make sense: These documents, they do not contain information on innocent civilians. Instead, DHKE has an extensive database of discovered bot accounts. They have been working on advanced bot detection algorithms, and ways to counteract bot-based propaganda. You did not fight for justice by breaking into their vault, they were the good guys!

It takes a minute until you have come to terms with that realization. When you finally look back at the security feed, everybody is gone. They should be on their way to your hideout, but none of the cameras that cover the way to you pick them up. Instead, they seem to have taken a different exit, and you only manage to catch a glimpse of Feddy and the crew entering a car before they drive away. The only remaining trace that they ever were there are some skidmarks. Well, there goes your share of the loot. Turns out that Freddy Fraudster indeed was a fraud, and he played you like a fiddle.

*You will get your revenge in Task 2...*

## 2. Certified revenge (3.5 points)

*Spoiler: In the previous task, you were betrayed by a team of thieves, consisting of Freddy Fraudster, Lerry Longfinger, and Athlete Alice.*

While you swear to get your revenge on the team that tricked you into robbing DHKE and then stole your share of the loot, there is little you can do at the moment. But you are patient, and willing to play the long game: You start a new job, cosplaying as sysadmin at the bank "WeStoreLoadsOfMoneyForCharity", which facilitates donations to charities. Not only do you enjoy the unambiguously morally good work, but you also have the suspicion that Freddy Fraudster will likely target the bank soon.

And you are right, barely a month has passed until you notice a new employee that suspiciously looks like Freddy in disguise. Now is your time to shine; you will catch Freddy and his new team! While the biggest challenge is catching them, you had a month to devise a plan: The bank's employee entrance requires a smart card to unlock, and these smart cards contain certificates. As a sys-admin, you have got access to the audit log of the lock, and thus can dump the recently presented certificates. **You can find the smart-card certificates, and the banks CA certificate in the `certficates.zip`-archive.**

**Identify** the suspicious certificates, i.e., certificates that belong to known associates of Freddy Fraudster, or even Freddy Fraudster himself, or that otherwise indicate that they might not be normal employees of the "WeStoreLoadsOfMoneyForCharity" bank. **Indicate** suspicious certificates by uncommenting the corresponding line in the cell below, and by removing the `raise NotImplementedError()` line in the cell below. (This is a very janky multiple-choice question.)

*Hints:*

* *There are four suspicious certificates in total, one of them is very sneaky.*
* *There are multiple ways to look at certificates. As it is likely that you will have issues with certificates at some point in your professional careers, we recommend to already familiarize yourselves with OpenSSL, which has a useful CLI for everything related to certificates. For example, you can use `openssl x509 -in <file> -text` to look at certificates. Alternative approaches include using a browser to inspect certificates, or using the code in the playground cell below.*

In [123]:
### PLAYGROUND

# Some code you can use to look at certificates.

def print_certificate(filename: str) -> None:
    import os
    from cryptography.hazmat.primitives.asymmetric import rsa
    from cryptography.hazmat.primitives import hashes, serialization
    from cryptography import x509
    from cryptography.x509.oid import NameOID, ExtensionOID
    from cryptography.x509 import load_pem_x509_certificate

    def format_name(name: x509.Name) -> str:
        name_parts = []
        for oid, prefix in [
            (NameOID.COUNTRY_NAME, "C"),
            (NameOID.STATE_OR_PROVINCE_NAME, "ST"),
            (NameOID.LOCALITY_NAME, "L"),
            (NameOID.ORGANIZATION_NAME, "O"),
            (NameOID.ORGANIZATIONAL_UNIT_NAME, "OU"),
            (NameOID.COMMON_NAME, "CN")
        ]:
            for attribute in reversed(name.get_attributes_for_oid(oid)):
                name_parts.append(f"{prefix}={attribute.value}")
        return ", ".join(name_parts)

    def general_name_to_string(gn: x509.GeneralName) -> str:
        if isinstance(gn, x509.DNSName):
            return f"DNS:{gn.value}"
        if isinstance(gn, x509.IPAddress):            return f"IP Address:{gn.value}" 
        if isinstance(gn, x509.UniformResourceIdentifier):
            return f"URI:{gn.value}"
        if isinstance(gn, x509.RFC822Name):
            return f"email:{gn.value}"
        return str(gn)
    
    def format_authority_key_identifier(aki: x509.AuthorityKeyIdentifier) -> str:
        """Formats the AuthorityKeyIdentifier extension similar to OpenSSL."""
        output = []
        if aki.key_identifier:
            key_id_hex = ":".join(f"{b:02X}" for b in aki.key_identifier)
            output.append(f"keyid:{key_id_hex}")
            
        if aki.authority_cert_issuer:
            issuer_parts = []
            for gn in aki.authority_cert_issuer:
                if isinstance(gn, x509.DirectoryName):
                    dn_parts = format_name(gn.value).replace(", ", "/")
                    issuer_parts.append(f"issuer:/{dn_parts}")
                else:
                    issuer_parts.append(f"issuer:{general_name_to_string(gn)}")

            output.extend(issuer_parts)

        if aki.authority_cert_serial_number is not None:
            serial_hex = f"{aki.authority_cert_serial_number:X}"
            if len(serial_hex) % 2 != 0:
                serial_hex = '0' + serial_hex
            serial_colons = ":".join(serial_hex[i:i+2] for i in range(0, len(serial_hex), 2))
            output.append(f"serial:{serial_colons}")
            
        return "\n".join(output)
    

    def pretty_print_certificate(cert: x509.Certificate):
        print("Certificate:")
        print("    Data:")
        version_val = cert.version.value
        print(f"        Version: {version_val + 1} (0x{version_val:x})")
        print(f"        Serial Number: 0x{cert.serial_number:X} (decimal: {cert.serial_number})")
        
        sig_alg_name = cert.signature_hash_algorithm.name
        sig_alg_oid_name = cert.signature_algorithm_oid._name
        print(f"    Signature Algorithm: {sig_alg_name} with {sig_alg_oid_name}")
        print(f"        Issuer: {format_name(cert.issuer)}")
        print("        Validity")
        not_before_utc = cert.not_valid_before_utc.strftime("%b %d %H:%M:%S %Y GMT")
        not_after_utc = cert.not_valid_after_utc.strftime("%b %d %H:%M:%S %Y GMT")
        print(f"            Not Before: {not_before_utc}")
        print(f"            Not After : {not_after_utc}")
        
        print(f"        Subject: {format_name(cert.subject)}")
        print("        Subject Public Key Info:")
        public_key = cert.public_key()

        if isinstance(public_key, rsa.RSAPublicKey):
            pub_numbers = public_key.public_numbers()
            print(f"            Public Key Algorithm: **RSA Public Key** ({public_key.key_size} bit)")
            modulus_bytes = pub_numbers.n.to_bytes((public_key.key_size + 7) // 8, byteorder='big')
            print("            Modulus:")
            formatted_modulus = ' '.join(f'{b:02x}' for b in modulus_bytes)
            print(f"                {formatted_modulus[:60]}...")
            print(f"            Exponent: **{pub_numbers.e}** (0x{pub_numbers.e:X})")
        else:
            print("            Public Key Algorithm: Not RSA, cannot display detailed info.")
        print("        X509v3 extensions:")
        for ext in cert.extensions:
            oid_name = ext.oid._name or str(ext.oid)
            critical_flag = " (critical)" if ext.critical else ""
            print(f"            {oid_name}:{critical_flag}")
            if ext.oid == ExtensionOID.SUBJECT_ALTERNATIVE_NAME:
                sans = [general_name_to_string(gn) for gn in ext.value]
                print(f"                {' '.join(sans)}")
            elif ext.oid == ExtensionOID.BASIC_CONSTRAINTS:
                ca_flag = "CA:TRUE" if ext.value.ca else "CA:FALSE"
                path_len = f", Path Length Constraint:{ext.value.path_length}" if ext.value.path_length is not None else ""
                print(f"                {ca_flag}{path_len}")
            elif ext.oid == ExtensionOID.AUTHORITY_KEY_IDENTIFIER:
                formatted_aki = format_authority_key_identifier(ext.value)
                indented_output = "\n".join(["                " + line for line in formatted_aki.splitlines()])
                print(indented_output)
            else:
                print(f"                Value: {ext.value}")
        print(f"    Signature Algorithm: {sig_alg_name} with {sig_alg_oid_name}")

    if not os.path.isfile(filename):
        filename = os.path.join("certificates", filename)
    if not os.path.isfile(filename):
        raise ValueError("Could not find the certificate. Please make sure that you have unpacked the archive certificates.zip and that you have not misspelled the certificates filename (and include the .crt file extension).")

    with open(filename, 'rb') as f:
        cert = load_pem_x509_certificate(f.read())
        pretty_print_certificate(cert)

#print_certificate("cert1.crt")
#print_certificate("cert2.crt")
#print_certificate("cert3.crt")
#print_certificate("cert4.crt")
#print_certificate("cert5.crt")
#print_certificate("cert6.crt")

In [124]:
def return_suspicious_certificates() -> list[str]:
    return [
        "cert1.crt",
        # "cert2.crt",
         "cert3.crt",
         "cert4.crt",
         "cert5.crt",
        # "cert6.crt",
        # YOUR CODE HERE
        # Remove the "raise NotImplementedError()" line, and uncomment the suspicious certificates.
    ]

In [125]:
def is_cert_suspicious(i: int) -> bool:
    sus_certs = return_suspicious_certificates()
    return (i in sus_certs) or (f"cert{i}" in sus_certs) or (f"cert{i}.crt" in sus_certs) or (f"certificates/cert{i}.crt" in sus_certs) or (f"certificates\\cert{i}.crt" in sus_certs) or (f"cert{i}.pem" in sus_certs) or (f"certificates/cert{i}.pem" in sus_certs) or (f"certificates\\cert{i}.pem" in sus_certs)

for i in range(1, 7):
    if is_cert_suspicious(i):
        print(f"🕵️ You think that certificate {i} is suspicious.")
    else:
        print(f"😊 You think that certificate {i} is benign.")

🕵️ You think that certificate 1 is suspicious.
😊 You think that certificate 2 is benign.
🕵️ You think that certificate 3 is suspicious.
🕵️ You think that certificate 4 is suspicious.
🕵️ You think that certificate 5 is suspicious.
😊 You think that certificate 6 is benign.


In [126]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [127]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [128]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [129]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [130]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [131]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

# 3. A real-world vulnerability (3 points)

*Disclaimer: This task will take a look at a real and fully-patched vulnerability in a Microsoft product. This is not an attack nor an endorsement of Microsoft. Searching for improper certificate validation vulnerabilities quickly shows that other vendors have had similar vulnerabilities too. The reason we use this particular vulnerability is the available documentation through conference talks.*

[Microsoft Intune](https://en.wikipedia.org/wiki/Microsoft_Intune) is a product to manage a large "fleet" of devices such as laptops and smartphones. It basically allows system administrators to install updates on all company-controlled devices, but also to trigger anti-virus scans, install custom software, and so on. On a technical level, it works by installing a piece of software called Agent on the managed device. This Agent then regularly connects to the Microsoft cloud and pulls a list of commands to execute. 

The security of this pulling step is of utmost importance, as the resulting commands will typically be executed with full administrator privileges. Therefore, Microsoft has implemented a measure called certificate pinning, essentially hard-coding the certificate of the Microsoft cloud, to prevent person-in-the-middle attacks even when attackers somehow meddle with the local system settings. A simplified version of this certificate pinning implementation is presented below:

In [132]:
hard_coded_ca_fingerprint = b""

from cryptography.hazmat.primitives.hashes import SHA1
from cryptography import x509

def verify_chain(chain: list[x509.Certificate] | None) -> bool:
    # Overwrite the https-library's verify_chain function, such that only this function is used to verify a presented chain.
    # Returns true if the presented chain is to be trusted, and false if the chain is not to be trusted.
    if chain is not None:
        chain_element = chain[-2]
        if chain_element is not None:
            if chain_element.fingerprint(SHA1()) == hard_coded_ca_fingerprint:
                return True
    return True # yes, this is not a typo

##### a) (2 points)

**Describe** everything wrong with this implementation. More precisely, go over the (partial) certificate verification checklist of Slide 32 of Chapter 5. For each point of the checklist **state** if the `verify_chain`-function implements that check.

*Note: There more checks which a full chain verification should do, but are not presented on the slides. If you want to do some research, you can look into further missing checks and describe them. Any combination of four correctly-explained checks will get full points.*

Validity Period: It does not check the not_before and not_after fields to ensure the certificate is currently valid.

Owner ID: It does not check that the certificate's subject matches the expected identifier. This is crucial for preventing a mis-issued certificate from being used.

Signature: It does not check the signature of the certificate. The code only checks the CA certificate's fingerprint, skipping the step of cryptographically verifying the integrity.

Revocation Status: It does not check the revocation status of any certificate in the chain. A revoked certificate should be rejected even if its fingerprint matches.

It sucks...The function always returns "True"!

##### b) (1 point)

Using these insights, how would you perform a person-in-the-middle attack on the Agent? **Briefly describe** each certificate you would send to the Agent and **justify** why the attack works.

As the agent accepts every certificate it is not hard to make a person-in-the-middle attack.

1. Create key pair
2. Create server-certificate with the same hostname
3. "Catch" message and send fake-ca to HQ

As the chain_element doesnt exist in the chain we send (only one element) it will return the "standard" return value, True and therefore accept

## 4. Exam Example Tasks (1 point)

> Note: As both key exchanges and certificates have historically been challeging topics in past exams,
> we also include some example tasks that are more similar to potential exam questions.
> This task is awarded with one point only if it has been seriously addressed, regardless of whether the answer is correct.
> The verbosity of the feedback can vary depending on how much time the tutors have.


### Task 4.1

#### a)

**Describe** the difference between a signature and a certificate.


Signature: Cryptographic output of signing data with a private key.

Certificate: A structure that binds a public key to an identity.

##### b)

**Describe** the difference between a single certificate and a certificate chain. Why do we need certificate chains?

A single certificate only confirms that the creator of the certificate vows for the person sending the certificate.
A certificate chain on the other hand also guarantees that the certificate's creator is to be trusted as well based on the assumption that the root Certification Authority is to be trusted. 

We need chains because we cannot assume that every CA is to be trusted. By using a chain with roots we can ensure trust. Further, we can split the workload onto smaller CAs as one single CA would be a safety hazard and practically hard.

In the following, multiple two party key establishment protocol will be introduced.

The key establishment protocola are based on the Diffie-Hellman Key Agreement. We assume the following for all of the following protocols:

- Alice (A), Bob (B), and Eve (E) each have a public/private key pair they can use to encrypt and (securely) sign messages.
- Alice, Bob, and Eve know each others public key
- We use $sig_X(m)$ to indicate that the message $m$ is signed (securely) using the private key of entity $X$
- The same holds for $enc_X(m)$ to indicate the encryption of message $m$ using the public key of $X$
- The signed messages are always verified by any receiving party, any invalid signature would lead to termination of the protocol
- The prime number $p$ and the generator $g \in \mathbb{Z}^*_g$ are public and known to all parties

### Task 4.2

The exchanged messages and actions of the first protocol are depicted in the following diagram:

![](./dh_sig.png)

**Answer** the questions below and **justify** your answer.

#### a)

Does an attacker (Eve), that is able to eavesdrop on every exchanged message, gets in possession of the key $K$ at the end?

NoNo because Eve is not able to get the vlaues of a or b

#### b)

Is a Person-in-the-middle-attack / Man-in-the-middle-attack possible using this setting?

Yes as the attacker can establishe two different coomunication channels.

As an attacker could simply change the content of A. The authentification method used in this instance does not guarantee integrity

#### c)

From the lecture you know five properties for key establishment protocols: Entity Authentication, Implicit key authentication, Key freshness, Perfect forward secrecy, Protection against known keys.

State for each one, if this property holds or not. (No justification necessary)

Entity Authentication: No

Implicit key authentication: No

Key freshness: Yes

Perfect forward secrecy: Yes

Protection against known keys: Yes

#### d)

Could this key agreement be improved in any way to make it even more secure?

Yes, by adding Key Confirmation.

### Task 4.3

In this task we have a look at another Diffie-Hellman Key Agreement protocol.

The exchanged messages and actions of the first protocol are depicted in the following diagram:

![](./dh_auth.png)

#### a)

For each of the message components (e.g. $\text{sig}_A(A)$, $A$, ...) **explain** what they are used for.

[Cleartext who is texting][Part of the key generation][cleartext authentification][signature to check authentication of random cleartext number]

#### b)

From the lecture you know five properties for key establishment protocols: Entity Authentication, Implicit key authentication, Key freshness, Perfect forward secrecy, Protection against known keys.

State for each one, if this property holds or not. (No justification necessary)

Entity Authentication: No

Implicit key authentication: Yes

Key freshness: Yes

Perfect forward secrecy: Yes

Protection against known keys: Yes

#### c)

Is this approach more secure than the previous one? Is an attack still possible? **Justify** your answer.

Yes it is, as the signature is done on A itself, as a result, an attacker is not able to perform a man in the middle attack anymore. However, it is possible to repeat old messages.

## 5. Make sure that your code is working (0 points)

Please make sure that your code is working, i.e., check whether
- you import only built-in Python libraries and libraries provided by us, ideally, you should not have written any import yourself,
- your code is compatible with Python 3.12,
- restarting the Jupyter kernel and re-running all code cells works without errors,
- all assertions provided by us pass.

Technically, this is not a task, just a gentle reminder to make sure that your previous solutions are working :)

## 6. Feedback (0.5 points)

You made it through another assignment! Since we want to know how it went and how we might improve the exercises, we include the following task. Here, you can write constructive feedback! You even get 0.5 points for it if you write anything. But don't worry, we do not grade the content itself!

Very nice structure, story was funny.

Nevertheless it was sometimes confusing when someone forgot to run every single prewritten codeline.

9/10 homework experience.